# Trabajo Práctico N° 6
<br/>

## Curso de Especialización en Inteligencia Artificial
## Visión por computadora 1
<br/>

### Autor: Maximiliano Torti
### Fecha: 19/06/21
<br/>

#### Enunciado:<br/>
Utilizando la webcam implementar un algoritmo utilizando filtros de Haar en
cascada que
- Detecte Rostros
- Dentro de los rostros detecte ojos
- Dentro de los rostros detecte sonrisas

In [1]:
import numpy as np
import cv2 as cv

In [2]:
# Leemos los clasificadores pre-entrenados (instalados en cv2/data/haarcascades/)
#--------------------------------------------------------------------------------
face_cascade = cv.CascadeClassifier('C:\\Users\MaxiT\\.conda\\envs\\CEIAIIA\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('C:\\Users\MaxiT\\.conda\\envs\\CEIAIIA\\Library\\etc\\haarcascades\\haarcascade_eye.xml')
smile_cascade = cv.CascadeClassifier('C:\\Users\MaxiT\\.conda\\envs\\CEIAIIA\\Library\\etc\\haarcascades\\haarcascade_smile.xml')

In [3]:
funny_eyes_img = cv.imread('funny_eyes.png',cv.IMREAD_UNCHANGED)
funny_eyes = False

funny_smile_img = cv.imread('funny_smile.png',cv.IMREAD_UNCHANGED)
funny_smile = False

# Obtengo stream de datos de la webcam
cap = cv.VideoCapture(0)
# Chequeo que la camara este disponible
if not cap.isOpened():
    raise IOError("WebCam no disponible")

while True:
    # Leo frame de la webcam
    ret, frame_orig = cap.read()
    
    # Pasamos la imagen a escala de grises
    if ret == True:
        grayImage = cv.cvtColor(frame_orig, cv.COLOR_BGR2GRAY)
        grayImage = cv.equalizeHist(grayImage)
        faces = face_cascade.detectMultiScale(grayImage, 1.1, 5)
    
        for (x,y,w,h) in faces:
            
            cv.rectangle(frame_orig,(x,y),(x+w,y+h),(255,255,0),2)
            
            roi_gray = grayImage[y:y+h, x:x+w] 
            roi_color = frame_orig[y:y+h, x:x+w] 
        
            eyes = eye_cascade.detectMultiScale(roi_gray,1.02,50)
            for (ex,ey,ew,eh) in eyes:
                if funny_eyes: 
                    funny_eyes_resized = cv.resize(funny_eyes_img, (ew,eh), \
                                                    interpolation = cv.INTER_AREA)
                    alpha_resized = funny_eyes_resized[:,:,3] / 255.0
                    alpha_resized = alpha_resized[:,:,np.newaxis]
                    funny_eyes_resized = funny_eyes_resized[:,:,0:3]
                    roi_color[ey:(ey+eh),ex:(ex+ew)]= \
                                                     alpha_resized * funny_eyes_resized + \
                                                     (1-alpha_resized) * roi_color[ey:(ey+eh),ex:(ex+ew)]
                else:
                    cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                
            smiles = smile_cascade.detectMultiScale(roi_gray,1.2,100)
            for (ex,ey,ew,eh) in smiles:
                if funny_smile: 
                    funny_smile_resized = cv.resize(funny_smile_img, (ew,eh), \
                                                    interpolation = cv.INTER_AREA)
                    alpha_resized = funny_smile_resized[:,:,3] / 255.0
                    alpha_resized = alpha_resized[:,:,np.newaxis]
                    funny_smile_resized = funny_smile_resized[:,:,0:3]
                    roi_color[ey:(ey+eh),ex:(ex+ew)]= \
                                                     alpha_resized * funny_smile_resized + \
                                                     (1-alpha_resized) * roi_color[ey:(ey+eh),ex:(ex+ew)]
                else:
                    cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,0,255),2)
    
        cv.imshow('Input',frame_orig)
    
    c = cv.waitKey(1)

    # Easter Egg - Funny eyes
    if c == 101:
        funny_eyes = not (funny_eyes)

    # Easter Egg - Funny smile
    if c == 115:
        funny_smile = not (funny_smile)

    # Finalizar con escape 
    if c == 27:
        break

cap.release()
cv.destroyAllWindows()